## Import Libraries

In [ ]:
#OS libs
import os
import shutil
import itertools
import pathlib
from PIL import Image

#Data handling tools
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
sns.set_style('whitegrid')
from sklearn.metrics import confusion_matrix , classification_report

#Deep learning tools
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Flatten , Conv2D , MaxPooling2D , Dropout , Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam , Adamax
from tensorflow.keras import regularizers

#Warnings
import warnings
warnings.filterwarnings('ignore')

## Prepare data and load into DataFrame

In [ ]:
train_data_path = '/kaggle/input/indian-food-images-12-different-dishes/Indian Cuisine/train'

filepaths = []
labels = []

folds = os.listdir(train_data_path)

for fold in folds:
    
    f_path = os.path.join(train_data_path , fold)
    imgs = os.listdir(f_path)
    
    for img in imgs:
        img_path = os.path.join(f_path , img)
        
        filepaths.append(img_path)
        labels.append(fold)
        
fseries = pd.Series(filepaths , name = 'Filepaths')
lseries = pd.Series(labels , name = 'Labels')

train_df = pd.concat([fseries , lseries] , axis = 1)

In [ ]:
train_df

In [ ]:
test_data_path = '/kaggle/input/indian-food-images-12-different-dishes/Indian Cuisine/test'

filepaths = []
labels = []

folds = os.listdir(test_data_path)

for fold in folds:
    
    f_path = os.path.join(test_data_path , fold)
    imgs = os.listdir(f_path)
    
    for img in imgs:
        img_path = os.path.join(f_path , img)
        
        filepaths.append(img_path)
        labels.append(fold)
        
fseries = pd.Series(filepaths , name = 'Filepaths')
lseries = pd.Series(labels , name = 'Labels')

test_df = pd.concat([fseries , lseries] , axis = 1)

In [ ]:
test_df

In [ ]:
val_data_path = '/kaggle/input/indian-food-images-12-different-dishes/Indian Cuisine/val'

filepaths = []
labels = []

folds = os.listdir(val_data_path)

for fold in folds:
    
    f_path = os.path.join(val_data_path , fold)
    imgs = os.listdir(f_path)
    
    for img in imgs:
        img_path = os.path.join(f_path , img)
        
        filepaths.append(img_path)
        labels.append(fold)
        
fseries = pd.Series(filepaths , name = 'Filepaths')
lseries = pd.Series(labels , name = 'Labels')

val_df = pd.concat([fseries , lseries] , axis = 1)

In [ ]:
val_df

## Image Data Generator


In [ ]:
img_size = (224 , 224)
batches = 32
img_shape = (img_size[0] , img_size[1] , 3)

tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

train_gen = tr_gen.flow_from_dataframe(train_df , x_col = 'Filepaths' , y_col = 'Labels' , target_size = img_size ,
                                      class_mode = 'categorical' , color_mode = 'rgb' , shuffle = True , batch_size = batches)
test_gen = ts_gen.flow_from_dataframe(test_df , x_col = 'Filepaths' , y_col = 'Labels',target_size = img_size ,
                                     class_mode = 'categorical' , color_mode = 'rgb' , shuffle = False , batch_size = batches)
valid_gen = ts_gen.flow_from_dataframe(val_df , x_col = 'Filepaths' , y_col = 'Labels' , target_size= img_size,
                                      class_mode = 'categorical' , color_mode = 'rgb' , shuffle = True, batch_size = batches)

In [ ]:
# Calculate class counts
class_counts = train_df['Labels'].value_counts().reset_index()
class_counts.columns = ['Class', 'Count']

# Calculate the total number of images in train_df
total_images = len(train_df)

# Calculate the percentage for each class based on the total number of images
class_counts['Percentage'] = (class_counts['Count'] / total_images) * 100

# Create the bar chart using Plotly Express
fig = px.bar(class_counts, x='Class', y='Percentage',
             title='Class Balance (Percentage)',
             labels={'Class': 'Class Label', 'Percentage': 'Percentage (%)'},
             color='Class', 
             height=600)

# Customize the layout
fig.update_layout(xaxis_title='Class Label', yaxis_title='Percentage (%)')
fig.show()

## Show Sample From Train Data

In [ ]:
g_dict = train_gen.class_indices
classes = list(g_dict.keys())
images , labels = next(train_gen)
num_samples = len(images)

plt.figure(figsize = (20, 20))

for i in range(min(16 , num_samples)):
    plt.subplot(4,4, i+1)
    image = images[i] / 255
    plt.imshow(image)
    index = np.argmax(labels[i])
    class_name = classes[index]
    plt.title(class_name , color = 'blue' , fontsize = 12)
    plt.axis('off')
plt.show();

## Model Structure

In [ ]:
base_model = tf.keras.applications.xception.Xception(weights= 'imagenet' ,include_top = False , input_shape = img_shape ,
                                                     pooling = 'max' )
num_classes = len(classes)
model = Sequential([
    base_model,
    BatchNormalization(axis = -1 , momentum = 0.99 , epsilon = 0.001),
    Dense(256, kernel_regularizer = regularizers.l2(l= 0.016) , activity_regularizer = regularizers.l1(0.006),
         bias_regularizer= regularizers.l1(0.006) , activation = 'relu'),
    Dropout(rate = 0.4 , seed = 40),
    Dense(num_classes , activation= 'softmax' )
])
model.compile(Adamax(learning_rate = 0.001) , loss = 'categorical_crossentropy' , metrics= ['accuracy'])
model.summary()

In [ ]:
history = model.fit(x= train_gen , validation_data= valid_gen , epochs= 20 , verbose = 1 , validation_steps = None , shuffle = False)

In [ ]:
train_acc = history.history['accuracy']
train_loss = history.history['loss']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]

index_acc = np.argmax(val_acc)
val_highest = val_acc[index_acc]

Epochs = [i+1 for i in range(len(train_acc))]

loss_label = f'Best Epoch = {str(index_loss + 1)}'
acc_label = f'Best Epoch = {str(index_acc + 1)}'

plt.figure(figsize = (20,8))
plt.style.use('fivethirtyeight')

plt.subplot(1,2,1)
plt.plot(Epochs ,train_loss , 'r' , label = 'Training Loss')
plt.plot(Epochs ,val_loss , 'g' , label = 'Validation Los')
plt.scatter(index_loss + 1 ,val_lowest , s = 150 , c = 'blue' , label = loss_label )
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(Epochs , train_acc , 'r' , label= 'Training Accuracy')
plt.plot(Epochs , val_acc , 'g' , label= 'Validation Accuracy')
plt.scatter(index_acc +1 , val_highest , s= 150 , c = 'blue' , label = acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show();

## Model Evaluation

In [ ]:
train_score = model.evaluate(train_gen ,steps= 8 , verbose = 1)
val_score = model.evaluate(valid_gen ,steps= 8 , verbose = 1)
test_score = model.evaluate(test_gen ,steps= 8 , verbose = 1)

In [ ]:
print(f'Train Loss = {round(train_score[0],5)} and Train accuracy = {train_score[1]}')
print(f'Validation Loss = {round(val_score[0],5)} and Validation accuracy = {val_score[1]}')
print(f'Test Loss = {round(test_score[0],5)} and Test accuracy = {test_score[1]}')

## Model Prediction

In [ ]:
preds = model.predict_generator(test_gen)

y_pred = np.argmax(preds, axis = 1)

## Confusion Matrix and Classification Report

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test_gen.classes, y_pred)

plt.figure(figsize= (10, 10))
plt.imshow(cm, interpolation= 'nearest', cmap= plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation= 45)
plt.yticks(tick_marks, classes)


thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')

plt.tight_layout()
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

plt.show()

In [ ]:
#Classification report
print(classification_report(test_gen.classes , y_pred , target_names = classes))